# Bibliotecas

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

# Problema
Temos um conjunto de produtos e suas descrições, e queremos recomendar produtos semelhantes ao usuário uma vez que ele esteja na página daquele produto.

## Dados Base

In [ ]:
products = [
    {
        "name": "Noise-Cancelling Headphones",
        "desc": "Wireless over-ear noise-cancelling headphones with 30-hour battery life."
    },
    {
        "name": "Bluetooth Speaker",
        "desc": "Portable Bluetooth speaker with rich bass and 10-hour playtime."
    },
    {
        "name": "Wireless Earbuds",
        "desc": "In-ear wireless earbuds with noise isolation and charging case."
    },
    {
        "name": "Gaming Headset",
        "desc": "Over-ear gaming headset with surround sound and noise-cancel mic."
    },
    {
        "name": "Smartwatch",
        "desc": "Water-resistant smartwatch with heart-rate monitor and GPS."
    },
    {
        "name": "Portable Charger",
        "desc": "10 000 mAh USB-C portable charger with fast-charging support."
    },
    {
        "name": "Wireless Keyboard",
        "desc": "Compact Bluetooth keyboard with quiet scissor-switch keys."
    },
    {
        "name": "4K Webcam",
        "desc": "1080p USB webcam with built-in microphone and low-light correction."
    },
    {
        "name": "USB-C Hub",
        "desc": "Multi-port USB-C hub with HDMI, Ethernet, USB-A, and SD card reader."
    },
    {
        "name": "E-Reader",
        "desc": "6-inch glare-free e-ink e-reader with adjustable warm light and 8 GB storage."
    },
]

# Solução (Busca por Similaridade)

## Carregando Modelo
Escolhemos o `all-miniLM-L6-v2` pois é pequeno e funciona bem para textos em inglês.

In [ ]:
# Setando o modelo
model_name = "all-MiniLM-L6-v2"
embedder = SentenceTransformerEmbeddings(model_name=model_name)

## Construindo o Banco Vetorial

### Separando Informações

In [ ]:
# Capturando somente as descrições
texts = [product["desc"] for product in products]

In [ ]:
for text in texts:
    print(text) # Mostrando os textos

In [ ]:
# Capturando os metadados
metadatas = [
    {
        "name": product["name"],
        "index": i
    }
    for i, product
    in enumerate(products)
]

In [ ]:
for metadata in metadatas:
    print(metadata) # Mostrando o índice e nome

### Criando Embeddings

In [ ]:
# Construindo com o FAISS no LangChain
vectordb = FAISS.from_texts(texts, embedding=embedder, metadatas=metadatas)

## Testando a Recomendação
Agora vamos assumir que estamos na página do produto 0 (Headphone com Cancelamento de Ruído). Podemos checar qual outro produto tem mais semelhança.

In [ ]:
# Capturando as informações da "página atual"
query_id = 0 # Índice da consulta
query_name = products[query_id]["name"]
query_text = products[query_id]["desc"]

In [ ]:
# Fazendo a busca por similaridade
k_recs = 3 # queremos os 3 mais semelhantes
raw_results = vectordb.similarity_search(query_text, k=k_recs+1) # + 1 pois o mais similar é ele próprio

In [ ]:
# Resultados que não são o próprio produto
results = [
    product
    for product
    in raw_results
    if product.metadata["name"] != query_name
]

In [ ]:
# Mostrando as recomendações
print(f"Recommendations for: {query_name}\n")
for rank, doc in enumerate(results, start=1):
    print(f"{rank}. {doc.metadata['name']} — {doc.page_content}")